<a href="https://colab.research.google.com/github/MashaKubyshina/Learning_to_code/blob/master/PART_3_Model_Interpretation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [93]:
# The code is from this tutorial https://towardsdatascience.com/text-classification-in-python-dd95d264c802

In [94]:
# In this project I selected Random Forest model to do preditions

In [95]:

import numpy as np
import random
import pickle
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import punkt
from nltk.corpus.reader import wordnet
from sklearn.feature_extraction.text import TfidfVectorizer

In [96]:
# we will load all Pickle files

# Dataframe
path_df = "/content/drive/MyDrive/Pickles/df.pickle"
with open(path_df, 'rb') as data:
    df = pickle.load(data)
    
# X_train
path_X_train = "/content/drive/MyDrive/Pickles/X_train.pickle"
with open(path_X_train, 'rb') as data:
    X_train = pickle.load(data)

# X_test
path_X_test = "/content/drive/MyDrive/Pickles/X_test.pickle"
with open(path_X_test, 'rb') as data:
    X_test = pickle.load(data)

# y_train
path_y_train = "/content/drive/MyDrive/Pickles/y_train.pickle"
with open(path_y_train, 'rb') as data:
    y_train = pickle.load(data)

# y_test
path_y_test = "/content/drive/MyDrive/Pickles/y_test.pickle"
with open(path_y_test, 'rb') as data:
    y_test = pickle.load(data)

# features_train
path_features_train = "/content/drive/MyDrive/Pickles/features_train.pickle"
with open(path_features_train, 'rb') as data:
    features_train = pickle.load(data)

# labels_train
path_labels_train = "/content/drive/MyDrive/Pickles/labels_train.pickle"
with open(path_labels_train, 'rb') as data:
    labels_train = pickle.load(data)

# features_test
path_features_test = "/content/drive/MyDrive/Pickles/features_test.pickle"
with open(path_features_test, 'rb') as data:
    features_test = pickle.load(data)

# labels_test
path_labels_test = "/content/drive/MyDrive/Pickles/labels_test.pickle"
with open(path_labels_test, 'rb') as data:
    labels_test = pickle.load(data)
    
# Random Forest Model
path_model = "/content/drive/MyDrive/Pickles/Models/best_rfc.pickle"
with open(path_model, 'rb') as data:
    rfc_model = pickle.load(data)
    
# Category mapping dictionary
category_codes = {
    'Computer/Tech': 0,
    'Engineering/Architecture': 1,
    'Business/Consulting': 2
}

category_names = {
    0: 'Computer/Tech',
    1: 'Engineering/Architecture',
    2: 'Business/Consulting'
}

In [97]:
# predictions on test set

predictions = rfc_model.predict(features_test)

In [98]:
# Test set dataframe with actual and predicted categories

# Indexes of the test set
index_X_test = X_test.index

# We get them from the original df
df_test = df.loc[index_X_test]

# Add the predictions
df_test['Prediction'] = predictions

# Clean columns
df_test = df_test[['job_description', 'category', 'Category_Code', 'Prediction']]

# Decode
df_test['Category_Predicted'] = df_test['Prediction']
df_test = df_test.replace({'Category_Predicted': category_names})

# Clean columns again
df_test = df_test[['job_description', 'category', 'Category_Predicted']]

In [99]:
# Let's check actual category and category predicted by RFC model

df_test.head()

,job_description,category,Category_Predicted
4005,"Job Description\n\nPerforms data analysis, int...",Computer/Tech,Computer/Tech
2457,TTEC requires all employees hired in the Unite...,Engineering/Architecture,Engineering/Architecture
357,At Actian we believe data should be used as a ...,Business/Consulting,Business/Consulting
1728,"Senior Data Scientist Duties Work with Retail,...",Business/Consulting,Business/Consulting
3909,Job Description\n\nWe are looking for a highly...,Computer/Tech,Engineering/Architecture


In [100]:
# Let's get missclassifed job_descriptions

condition = (df_test['category'] != df_test['Category_Predicted'])

df_misclassified = df_test[condition]

df_misclassified.head(3)

,job_description,category,Category_Predicted
3909,Job Description\n\nWe are looking for a highly...,Computer/Tech,Engineering/Architecture
888,AWS Data Scientist- Computer Software- Minneso...,Engineering/Architecture,Business/Consulting
2714,Credit Sesame is the nation’s only full-lifecy...,Engineering/Architecture,Computer/Tech


In [101]:
# Function to get the sample of 3 missclasifed job postings

def output_article(row_article):
    print('Actual Category: %s' %(row_article['category']))
    print('Predicted Category: %s' %(row_article['Category_Predicted']))
    print('-------------------------------------------')
    print('Text: ')
    print('%s' %(row_article['job_description']))

In [102]:
# get random number of indices

random.seed(8)
list_samples = random.sample(list(df_misclassified.index), 5)
list_samples

[2524, 781, 949, 714, 1778]

In [103]:
# Let's look at all 3 job postings

# first one


output_article(df_misclassified.loc[list_samples[0]])

Actual Category: Business/Consulting
Predicted Category: Computer/Tech
-------------------------------------------
Text: 
Look for more than answers.   At Quest, we are on a continuous journey of discovery and development. It’s this attitude that has made us an industry leader and the #1 Diagnostic Lab in the US. For those joining us, we offer exciting and fast-moving career opportunities where you can affect change at a rate unheard of in many organizations of our size and scope. While we invest in and develop technology to drive our innovations, our ongoing success relies on our people. As a Data Scientist, you will work for the world leader in the industry, with a career where you will have the opportunity to collaborate and affect change while expanding your leadership skills and technical knowledge. You can make a real impact in a market that is growing and developing. Data Scientist, the role:   The Data Scientist will support the franchises by using internal Quest Diagnostics da

In [104]:
# second missclasifed job description

output_article(df_misclassified.loc[list_samples[1]])

Actual Category: Business/Consulting
Predicted Category: Computer/Tech
-------------------------------------------
Text: 
Job Title: Data Analyst/Data Scientist Location: San Francisco, CA Duration: 3+ months   CTH Responsibilities: • Skilled at defining and asking key questions that help clients discover their needs, building trust by communicating effectively with clients about status and issues, developing and presenting convincing recommendations that address key client objectives, setting appropriate expectations with client and managing them throughout the project life cycle • Analyze industry trends, conducting discussions for insights, interpreting dataextensive healthcare repository, examining models and tools, formulating studies and making recommendations to stakeholders. Design, build and deploy BI solutions in Tableau and create ad-hoc reports as needed • Support all aspects of projects; including formulating research and analysis plans, developing strategies instrumental 

In [105]:
# third case

output_article(df_misclassified.loc[list_samples[2]])

Actual Category: Engineering/Architecture
Predicted Category: Computer/Tech
-------------------------------------------
Text: 
Read what people are saying about working here. 

Part-time, Internship

ENGIE Insight, formerly Ecova, partners with multi-site businesses that aim to thrive in a sustainable world. With a foundation of accurate and comprehensive resource data – including energy, water, waste and telecom – we apply technology and people expertise to lower costs, drive efficiencies and reduce environmental impact. More than 1,000 customers, including 25%+ of the Fortune 500, turn to ENGIE Insight to help move their sustainable resource management initiatives forward.

ENGIE Insight’s people are our greatest strength, bound by our vision to enable a sustainable world where people, organizations and the environment thrive. We are passionate about making our company and the world a better place; responsible in working with commitment and accountability; collaborative with our coll

In [106]:
# Fourth case

output_article(df_misclassified.loc[list_samples[3]])

Actual Category: Engineering/Architecture
Predicted Category: Computer/Tech
-------------------------------------------
Text: 
Read what people are saying about working here. 

The Applied Analytics COE team isresponsible for developing analytics-driven solutions to help GM Organizations achievetheir business goals. The individual will work as a member of amulti-disciplinary team of various experience levels that will be drivingsolution design, development, and deployment of analytic models in support ofbusiness facing analytic groups. The team is responsible for bothadvanced analytics strategy and applied, project-based solutions, focusing onthe company’s most critical business areas.

Responsibilities include thefollowing:

Work as a member of a cross-functional team to propose and implement high-impact data and analytic solutions that address business challenges across a variety of business units

Apply Data Science and Modeling techniques to drive data driven decision making and so

In [107]:
# fifth sample

output_article(df_misclassified.loc[list_samples[4]])

Actual Category: Business/Consulting
Predicted Category: Computer/Tech
-------------------------------------------
Text: 
Learn and work on meaningful initiatives with some of the best and brightest in the market research industry. The NPD Group provides the world’s most successful brands with leading market research, combining consumer and retail point-of-sale data with analytic solutions to interpret today’s market trends while anticipating tomorrow’s. In addition, we offer a career filled with innovation and growth to the forward-thinking problem solvers who join our team. Position Overview You’re a passionate, dynamic, and highly motivated individual who’s looking for an opportunity to have an impact on some of the most well-known companies in the world.  This integral member of the Research Sciences team will report directly to the Research Manager. If you enjoy working with clients to influence their business solutions, this job is for you. NPD’s Research Science Group works to e

**Lets test our model on real life job postings**

In [108]:
# Downloading stopwords list

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [109]:
# Loading the stop words in english

stop_words = list(stopwords.words('english'))

In [110]:
# download wordnet
)

nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [140]:
# Copy our setup from feature selection (part 2)
# Parameter election
ngram_range = (1,2)
min_df = 10
max_df = 1.
max_features = 300

# define and fit TF-IDF vectorizer

tfidf = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        max_features=max_features,
                        norm='l2',
                        sublinear_tf=True)
features_train = tfidf.fit_transform(X_train).toarray()
labels_train = y_train
print("Train features", features_train.shape)

features_test = tfidf.transform(X_test).toarray()
labels_test = y_test
print("Test features", features_test.shape)

Train features (1500, 300)
Test features (500, 300)


In [141]:
punctuation_signs = list("?:!.,;")
stop_words = list(stopwords.words('english'))

def create_features_from_text(text):
    
    # Dataframe creation
    lemmatized_text_list = []
    df = pd.DataFrame(columns=['job_description'])
    df.loc[0] = text
    df['Content_Parsed_1'] = df['job_description'].str.replace("\r", " ")
    df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("\n", " ")
    df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("    ", " ")
    df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace('"', '')
    df['Content_Parsed_2'] = df['Content_Parsed_1'].str.lower()
    df['Content_Parsed_3'] = df['Content_Parsed_2']
    for punct_sign in punctuation_signs:
        df['Content_Parsed_3'] = df['Content_Parsed_3'].str.replace(punct_sign, '')
    df['Content_Parsed_4'] = df['Content_Parsed_3'].str.replace("'s", "")
    wordnet_lemmatizer = WordNetLemmatizer()
    lemmatized_list = []
    text = df.loc[0]['Content_Parsed_4']
    text_words = text.split(" ")
    for word in text_words:
        lemmatized_list.append(wordnet_lemmatizer.lemmatize(word, pos="v"))
    lemmatized_text = " ".join(lemmatized_list)    
    lemmatized_text_list.append(lemmatized_text)
    df['Content_Parsed_5'] = lemmatized_text_list
    df['Content_Parsed_6'] = df['Content_Parsed_5']
    for stop_word in stop_words:
        regex_stopword = r"\b" + stop_word + r"\b"
        df['Content_Parsed_6'] = df['Content_Parsed_6'].str.replace(regex_stopword, '')
    df = df['Content_Parsed_6']
    #df = df.rename(columns={'Content_Parsed_6': 'Content_Parsed'})
    
    # TF-IDF
    features = tfidf.transform(df).toarray()
    
    return features

In [142]:
def get_category_name(category_id):
    for category, id_ in category_codes.items():    
        if id_ == category_id:
            return category

In [143]:
# let's try to predict a job posting category from a random job description

def predict_from_text(text):
    
    # Predict using the input model
    prediction_rfc = rfc_model.predict(create_features_from_text(text))[0]
    prediction_rfc_proba = rfc_model.predict_proba(create_features_from_text(text))[0]
    
    # Return result
    category_rfc = get_category_name(prediction_rfc)
    
    print("The predicted category using the RFC model is %s." %(category_rfc) )
    print("The conditional probability is: %a" %(prediction_rfc_proba.max()*100))


In [139]:
text="""

Position: Director, Demand Conversion Data Science

Hiring Leader: David Ralston, Senior Director Engagement Analytics

Business Unit: Digital Media

Location: Bay Area, CA preferred

Our company

Changing the world through digital experiences is what Adobe’s all about. We give everyone—from emerging artists to global brands—everything they need to design and deliver extraordinary digital experiences. We’re passionate about empowering people to create beautiful and powerful images, videos, and apps, and transform how companies interact with customers across every screen.

We’re on a mission to hire the very best and are committed to creating extraordinary employee experiences where everyone is respected and has access to equal opportunity. We realize that new ideas can come from everywhere in the organization, and we know the next big idea could be yours!

The challenge

Use your expertise in marketing sciences to drive the next stage of growth for Adobe! You will bridge TAM and prospect audiences to identify and nurture consumers on their journey towards becoming long-tenured customers. Understanding of traffic sources, landing pages, and customer touch-points for each customer to enable optimization as we move from prospect to visitor to customer.

This leader will work closely with paid media, content, campaign, and growth teams to deliver an outstanding experience for new and existing customers. A successful candidate will own the conversion funnel, including critical metrics, targets, and ongoing optimization. Also, the Director will develop a test framework, craft the testing roadmap, developing the personalization strategy, and executing the plan across every stage of our acquisition funnel.

The ideal candidate will have hands-on experience with data science applied to digital marketing of subscription products/services and a passion for developing deep customer understanding that will advise optimization and personalization strategies. People, technology, and data science leadership combined with a strong focus on working with a diverse set to teams to deliver business results are important elements of this leadership role.

What you’ll do
Lead development of the demand conversion funnel, including all aspects of acquisition, nurture, conversion and advocacy stages of the customer journey. Deep understanding of TAM and how to reach prospective customers. Quantify health of traffic, trials and conversion to optimize, set strategy and inform investment
Work with the GTM, Marketing and Campaign teams to strategize on the overall demand plan and build audiences for each event that ultimately translate to revenue.
Develop a robust, performance-based test & learn plan, with a prioritized focus on the biggest opportunities
Deliver innovative solutions to attribution and intent classification with the explicit goal of unlocking hidden opportunities for optimization.
Deliver ongoing insights into performance and opportunities (unlocks) to exec audience including partnering with GTM and Marketing leaders on pricing, promo, trial, up/down/cross sell strategies
Build critical metrics including free trial conversion rate, net new attach rates, up/cross/down-sell. You will have heavy influence over setting the target for these rates in the annual budget, and significant responsibility for ensuring that the company meets these targets.
What you need to succeed
10+ years of digital marketing & acquisition experience with a focus on performance marketing and demand conversion.
Experience building and leading a large team, with an ongoing focus on mentorship and career development of highly technical team. Ideally has led teams of 10-15+.
Advanced degree in related field with proven hands-on experience using diverse data to deliver conversion optimization
Highly analytical attitude with a deep understanding of acquisition funnel, economics and profitability levers
Strong leadership skills, with the ability to work optimally with multiple partners.
Subscription business model and/or digital product industry experience (agency or corporate) a plus

At Adobe, you will be immersed in an exceptional work environment that is recognized around the world. You will also be surrounded by colleagues who are committed to helping each other grow through our unique Check-In approach where ongoing feedback flows freely. If you’re looking to make an impact, Adobe's the place for you. Discover what our employees are saying about their career experiences on the Adobe Life blog and explore the meaningful benefits we offer.

Adobe is an equal opportunity employer. We hire hard working individuals, regardless of gender, race or color, ethnicity or national origin, age, disability, religion, sexual orientation, gender identity or expression, or veteran status. We know that when our employees feel appreciated and included, they can be more creative, innovative and successful. This is what it means to be Adobe For All. Learn more about our vision here.

We will ensure that individuals with disabilities are provided reasonable accommodation to participate in the job application or interview process, to perform essential job functions, and to receive other benefits and privileges of employment. Please contact us to request accommodation.


"""

predict_from_text(text)

The predicted category using the RFC model is Computer/Tech.
The conditional probability is: 55.07815950655625


In [144]:
text="""
Interested in being a part being a part of a multi-billion dollar international organization? Our Client is looking for a Data Scientist to work within their rapidly growing IoT Lab in the heart of Silicon Valley. This opportunity will be working on projects anywhere from home automation and autonomous vehicles to the connectivity between devices.

You will be working with different projects with a talented team of backend, mobile, and front end engineers. They are looking for someone who will work directly with executive teams of leading companies, getting great visibility within the company.

Required Skills & Experience
B.S./M.S. in Computer Science, Engineering or related
2-3+ years of Data Science / ML experience
Proficiency in Python ML Libraries / Packages
Experienced with cloud technology: AWS, EC2, & S3
Strong Design, Analytical Skills, and excellent communication skills
Experience developing large-scale distributed systems


Desired Skills & Experience
Experience building a data-processing pipeline is big plus

Benefits & Perks

Competitive Salary DOE, Early Stage Equity, Benefits

"""

predict_from_text(text)

The predicted category using the RFC model is Business/Consulting.
The conditional probability is: 49.82238349885409


In [145]:
text="""
Overview
As USAC continues to support universal service impacting health care providers, educators, and
consumers, it is critical now more than ever that we hire people equally passionate and
committed to helping fulfill our mission. We are currently seeking an experienced, dynamic and
collaborative professional to serve as Data Analyst within on our Data Strategy team. 
Reporting to the Senior Manager of Data Strategy and Insight you will:
 performs data reporting and analytics, including requirements gathering for reports,
aggregating and presenting information from various programmatic data sources for
monthly and quarterly reporting purposes.
This is an outstanding career opportunity for an individual interested in a genuine professional
challenge in support of a public-spirited mission.
Responsibilities
Working in a creative and fast-paced environment, the Data Analyst&#39;s responsibilities will
include the following:
 Analyze large sets of program data to identify opportunities to improve the efficiency,
integrity, and stakeholder experience of Universal Service program participants.
 Perform research and analysis using programmatic data, to relay programmatic impacts,
trends, patterns, statistics, etc. across the company.
 Recommend program process improvements based on data while maintaining reporting
standards while ensuring consistency and quality of deliverables.
 Create and maintain queries using SQL or other programming languages (with a
preference for Open Source programming languages such as R and Python) as necessary
for consistency and efficiency of data retrieval from various systems.
 Research methods and tools to create efficiency for reporting, GIS mapping, and other
analytical and computational processes.
 Maintain, develop, and implement policies and procedures that ensure the accuracy and
integrity of USAC program data and reporting.
 Participate in the evaluation of new technologies to ensure the technology architecture
and data analysis and visualization tools are in line with company’s evolving business
intelligence needs.
 Support managers, directors, and members of the leadership team in various projects,
as needed.
 Other specific duties as assigned.
 
About You
 
The successful candidate will excel at operating in a diverse and fluid environment, and will be
crucial for the success of the Date Strategy team.

 You possess Bachelor’s degree from an accredited institution in a related field.
 Three (3) to Four (4) years of professionally related experience.
 Experience in the data science field performing descriptive, predictive, and prescriptive
analysis and developing data products and tools that provide actionable insights and
strategic guidance to managers and people in leadership positions.
 Ability to work independently, but also collaborate and work across other departments
as needed.
 Proven experience working with data warehoused in databases using SQL and
programming languages for data science such as R or Python (or a demonstrated
capacity to learn them).
 Experience presenting complex analysis and results in plain English for a non-technical
audience using data visualizations (i.e. charts, tables, and graphs) on various software
platforms.
 Excellent written and verbal communication skills, are naturally curious and inquisitive,
and are interested in continuously improving our work.
 Excellent analytical skills with the ability to collect, organize, analyze, and disseminate
significant amounts of information with attention to detail and accuracy.
 Proficient with Microsoft Office Suite.

"""

predict_from_text(text)

The predicted category using the RFC model is Computer/Tech.
The conditional probability is: 46.186765176701755


In [146]:
# fun fact: I tried testing with a job description for food manager in hospitality and this is what I got :)

text="""

The Ingleside Hotel and Springs Water Park in Pewaukee is adding an experienced Food and Beverage Manager to our management team. If you have experience in banquet, restaurant or bar management and have strong leadership, organizational, communication, and guest service skills, this would be a perfect fit. You will help manage operations for our banquets, restaurant and water park snack bar. Position is primarily PM shift, including weekends and holidays. Must have (or obtain) a Waukesha bartending license.
We have a reputation for providing extraordinary service, impeccable facilities and superb cuisine. As the Food and Beverage Manager, you will be responsible for ensuring we are achieving that vision. We offer you a family atmosphere and supportive management. Full benefit package + fun perks! EOE/M/F/D/V/SO
"""

predict_from_text(text)

The predicted category using the RFC model is Business/Consulting.
The conditional probability is: 67.36168497190039
